In [2]:
import numpy as np
import json
from livecellx.core import (
    SingleCellTrajectory,
    SingleCellStatic,
    SingleCellTrajectoryCollection,
)

from livecellx.core.datasets import LiveCellImageDataset
from livecellx.preprocess.utils import (
    overlay,
    enhance_contrast,
    normalize_img_to_uint8,
)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from pathlib import Path
import pandas as pd

Loading Mitosis trajectory Single Cells

In [3]:
from livecellx.track.classify_utils import load_class2samples_from_json_dir, load_all_json_dirs
# sample_json_dir = Path("./EBSS_starvation_24h_xy16_annotation")

sample_json_dirs_v0 = [Path(r"./datasets/test_scs_EBSS_starvation/XY1/annotations"), Path(r"./datasets/test_scs_EBSS_starvation/XY16/annotations")]

round1_json_dirs = sample_json_dirs_v0 + [
    Path(r"../datasets/mitosis-annotations-2023/shiman_XY01/XY01"),
Path(r"../datasets/mitosis-annotations-2023/shiman_XY09/XY09"),
Path(r"../datasets/mitosis-annotations-2023/shiman_XY10/XY10"),
Path(r"../datasets/mitosis-annotations-2023/Yajushi/tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19/XY1/annotations"),
]

round2_json_dirs = [
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.5^4/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.75^4/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^3/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4/"),

] + [
    Path(f"../datasets/mitosis-annotations-2023/Gaohan_tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19/XY{pos}/annotations") for pos in range(4, 14)
]

round3_json_dirs = [
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_normal/0.25^4/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4/"),
    Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_normal/C10^3/")
]

sample_json_dirs = sample_json_dirs_v0 + round1_json_dirs + round2_json_dirs + round3_json_dirs
for sample_dir in sample_json_dirs:
    assert sample_dir.exists(), f"{sample_dir} does not exist"
all_class2samples, all_class2sample_extra_info = load_all_json_dirs(sample_json_dirs)
           

constructing single cells from json dict: 100%|██████████| 11/11 [00:00<00:00, 16103.78it/s]


Loaded 8 annotated samples from datasets/test_scs_EBSS_starvation/XY1/annotations/mitosis
Loaded 0 annotated samples from datasets/test_scs_EBSS_starvation/XY1/annotations/apoptosis
Loaded 0 annotated samples from datasets/test_scs_EBSS_starvation/XY1/annotations/normal


constructing single cells from json dict: 100%|██████████| 13/13 [00:00<00:00, 19806.01it/s]


Loaded 12 annotated samples from datasets/test_scs_EBSS_starvation/XY16/annotations/mitosis
Loaded 0 annotated samples from datasets/test_scs_EBSS_starvation/XY16/annotations/apoptosis
Loaded 0 annotated samples from datasets/test_scs_EBSS_starvation/XY16/annotations/normal


constructing single cells from json dict: 100%|██████████| 11/11 [00:00<00:00, 26439.74it/s]


...

Loaded 0 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/0.25^4/mitosis
Loaded 0 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/0.25^4/apoptosis
Loaded 5 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/0.25^4/normal


constructing single cells from json dict: 100%|██████████| 7/7 [00:00<00:00, 12325.83it/s]


Loaded 0 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4/mitosis
Loaded 0 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4/apoptosis
Loaded 11 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/C0.5^4/normal


constructing single cells from json dict: 100%|██████████| 7/7 [00:00<00:00, 13273.11it/s]

Loaded 0 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/C10^3/mitosis
Loaded 0 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/C10^3/apoptosis
Loaded 5 annotated samples from ../datasets/mitosis-annotations-2023/shiman_CXA_normal/C10^3/normal


In [4]:
len(all_class2samples["mitosis"]), len(all_class2samples["normal"])

(331, 21)

In [5]:
all_scs = []
for cls, class_samples in all_class2samples.items():
    for sample in class_samples:
        all_scs.extend(sample)

# Remove duplicates
all_scs = list(set(all_scs))

In [6]:
print("#scs:", len(all_scs))

#scs: 5105


## Generate normal cases

Split traintest sets

In [7]:
# Split all_scs into train and test cases:
from sklearn.model_selection import train_test_split
train_scs, test_scs = train_test_split(all_scs, test_size=0.2, random_state=42)

normal cases: input mask -> correct mask


In [14]:

import importlib
import livecellx
import livecellx.segment.ou_utils
importlib.reload(livecellx.segment.ou_utils)

def gen_csn_correct_case(scs, out_dir, filename_pattern="img-%d_scId-%s.tif"):
    out_subdir = out_dir / "correct_cases"
    raw_out_dir = out_subdir / "raw"
    seg_out_dir = out_subdir / "seg"
    gt_out_dir = out_subdir / "gt"
    gt_label_out_dir = out_subdir / "gt_label_mask"
    augmented_seg_dir = out_subdir / "augmented_seg"
    raw_transformed_img_dir = out_subdir / "raw_transformed_img"
    augmented_diff_seg_dir = out_subdir / "augmented_diff_seg"

    os.makedirs(raw_out_dir, exist_ok=True)
    os.makedirs(seg_out_dir, exist_ok=True)
    os.makedirs(gt_out_dir, exist_ok=True)
    os.makedirs(augmented_seg_dir, exist_ok=True)
    os.makedirs(gt_label_out_dir, exist_ok=True)
    os.makedirs(raw_transformed_img_dir, exist_ok=True)
    os.makedirs(augmented_diff_seg_dir, exist_ok=True)

    scale_factors = [0] # We 
    train_path_tuples = []
    augmented_data = []

    for sc in tqdm(scs):
        img_id = sc.timeframe
        seg_label = sc.id
        # (img_crop, seg_crop, combined_gt_label_mask) = underseg_overlay_gt_masks(seg_label, scs, padding_scale=2)
        img_crop = sc.get_img_crop()
        seg_crop = sc.get_contour_mask()
        # Only 1 gt mask for mask cases, seg_crop is sufficient
        combined_gt_label_mask = seg_crop

        filename = (filename_pattern % (img_id, seg_label))
        raw_img_path = raw_out_dir / filename
        seg_img_path = seg_out_dir / filename
        gt_img_path = gt_out_dir / filename
        gt_label_img_path = gt_label_out_dir / filename

        # call csn augment helper
        livecellx.segment.ou_utils.csn_augment_helper(img_crop=img_crop, 
            seg_label_crop=seg_crop, 
            combined_gt_label_mask=combined_gt_label_mask,
            scale_factors=scale_factors,
            train_path_tuples=train_path_tuples,
            augmented_data=augmented_data,
            img_id=img_id,
            seg_label=seg_label,
            gt_label=None,
            raw_img_path=raw_img_path,
            seg_img_path=seg_img_path,
            gt_img_path=gt_img_path,
            gt_label_img_path=gt_label_img_path,
            augmented_seg_dir=augmented_seg_dir,
            augmented_diff_seg_dir=augmented_diff_seg_dir,
            raw_transformed_img_dir=raw_transformed_img_dir,
            df_save_path=None,
            filename_pattern="img-%d_scId-%s.tif"
        )


    pd.DataFrame(
        train_path_tuples,
        columns=["raw", "seg", "gt", "raw_seg", "scale", "aug_diff_mask", "gt_label_mask", "raw_transformed_img"],
    ).to_csv(out_subdir / "data.csv", index=False)

In [13]:
len(test_scs)

1021

In [15]:
train_out_dir = Path("./notebook_results/a549_ccp_vim/train_correct_from_lca/")
test_out_dir = Path("./notebook_results/a549_ccp_vim/test_correct_from_lca/")
gen_csn_correct_case(train_scs, train_out_dir, filename_pattern="img-%d_scId-%s.tif")
gen_csn_correct_case(test_scs, test_out_dir, filename_pattern="img-%d_scId-%s.tif")


100%|██████████| 1021/1021 [00:26<00:00, 38.76it/s]


In [16]:
for sc in all_scs:
    dims = sc.get_img_crop().shape
    if dims[0] < 2 or dims[1] < 2:
        print(sc.id)

In [ ]:
# visualize augmented data on one row
for aug_data in augmented_data:
    img_id = aug_data["img_id"]
    seg_label = aug_data["seg_label"]
    img_crop = aug_data["img_crop"]
    seg_crop = aug_data["seg_crop"]
    combined_gt_label_mask = aug_data["combined_gt_label_mask"]
    aug_seg_crop = aug_data["aug_seg_crop"]
    aug_diff_mask = aug_data["aug_diff_mask"]
    combined_gt_label_mask = aug_data["combined_gt_label_mask"]
    raw_transformed_img_crop = aug_data["raw_transformed_img_crop"]
    
    fig, ax = plt.subplots(2, 4, figsize=(20, 10))
    ax[0, 0].imshow(img_crop)
    ax[0, 0].set_title("raw")
    ax[0, 1].imshow(seg_crop)
    ax[0, 1].set_title("seg")
    ax[0, 2].imshow(combined_gt_label_mask)
    ax[0, 2].set_title("gt_label")
    ax[0, 3].imshow(aug_seg_crop)
    ax[0, 3].set_title("aug_seg")
    ax[1, 0].imshow(aug_diff_mask)
    ax[1, 0].set_title("aug_diff_mask")
    ax[1, 1].imshow(raw_transformed_img_crop)
    ax[1, 1].set_title("raw_transformed_img")

    # show distribution of raw transformed image
    ax[1, 2].hist(raw_transformed_img_crop.flatten(), bins=100)
    ax[1, 2].set_title("raw_transformed_img hist")
    plt.show()
    break

## Generate oversegmentation cases

In [ ]:
class BackgroundSimulator:
    pass

class OverSegSimulator:
    pass

class UnderSegSimulator:
    pass


In [ ]:
a = np.array([[1, 2], [3, 4]])
a[(1,1), (0,0)]

In [ ]:
from copy import deepcopy
import random
from livecellx.segment.overseg_synthesis import divide_single_cell_brute_force,  gen_synthetic_overseg
from livecellx.trajectory.contour_utils import get_cellTool_contour_points, viz_contours


sampled_sc = single_cells[0]
# contour_points = sampled_sc.contour
# sampled_contour_points = contour_points[::3]
sc_img_path_substr = os.path.basename(sampled_sc.meta["path"]).split(".")[0]
seg_mask = seg_data.get_img_by_url(sc_img_path_substr, exact_match=False)
seg_crop = sampled_sc.gen_skimage_bbox_img_crop(sampled_sc.bbox, seg_mask)

sampled_sc.show_contour_mask()
plt.show()
# plt.imshow(seg_crop[:10, :])
# plt.show()

# print("contour points:", sampled_sc.get_contour_coords_on_img_crop())

# two test cases
sampled_points = [[ 85., 66.], [ 73., 0]]
# sampled_points = [[ 3., 50.], [ 3., 96.0]]
# sample_points = [[ 37., 120.], [54., 12.]]

contour_img = sampled_sc.get_contour_img()
divide_single_cell_brute_force(sampled_sc, seg_crop=contour_img, sampled_points=sampled_points)
# divide_single_cell_brute_force(sampled_sc, seg_crop=contour_img, sampled_points=None)
# get_line_pixels([1, 5], [10, 10])
# plt.imshow(seg_crop)
plt.imshow(contour_img)

Qualitatively generate and test bf more thoroughly...

In [ ]:
def test_divide_bf(sc):
    contour_img = sc.get_contour_img()
    divide_single_cell_brute_force(sc, seg_crop=contour_img, sampled_points=None)
    plt.imshow(contour_img)
    plt.show()

for i in range(3):
    test_divide_bf(sampled_sc)

In [ ]:
from scipy import ndimage
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.morphology import local_maxima, h_maxima
from skimage.measure import regionprops, label
import scipy.stats
import livecellx
from livecellx
from livecellx.segment.overseg_synthesis import divide_single_cell_watershed
from livecellx.segment.ou_utils import dilate_or_erode_label_mask

raw_crop = sampled_sc.get_contour_img()
# new_img, label_mask = divide_single_cell_watershed(sampled_sc, peak_distance=20, num_gauss_areas=4, marker_method="local", gauss_center_val=150)

new_raw_crop, label_mask, edt_distance, markers = divide_single_cell_watershed(sampled_sc, num_gauss_areas=3, marker_method="hmax", edt_gauss_center_val=10, gauss_std=16, h_threshold=1, return_all=True)



# import importlib
# importlib.reload(livecellx.preprocess.utils)
# importlib.reload(livecellx.segment.ou_utils)


eroded_label_mask = livecellx.segment.ou_utils.dilate_or_erode_label_mask(label_mask, scale_factor=-0.1, bg_val=0)

fig, axes = plt.subplots(1, 6, figsize=(60, 10))
ax = axes[0]
ax.imshow(raw_crop)
ax.set_title("raw_img")
ax = axes[1]
ax.imshow(new_raw_crop)
ax.set_title("new_img")
ax = axes[2]
ax.imshow(label_mask)
ax.set_title("label_mask")
ax = axes[3]
ax.imshow(edt_distance)
ax.set_title("edt_distance")
ax = axes[4]
ax.imshow(markers)
ax.set_title("markers")
ax = axes[5]
ax.imshow(eroded_label_mask)
ax.set_title("eroded_label_mask")
plt.show()

### Generate synthetic data by watershed

In [ ]:
overseg_uns_key = "overseg_imgs"
num_gauss_areas = np.arange(3, 6)
num_samples = 5

def process_sc_synthetic_overseg_crops(sc):
    sc.uns[overseg_uns_key] = []
    for num_gauss_area in num_gauss_areas:
        label_masks_and_params_hmax = gen_synthetic_overseg(sc, num_samples=num_samples, peak_distance=10, num_gauss_areas=num_gauss_area, marker_method="hmax", edt_gauss_center_val=10, gauss_std=16, h_threshold=1)
        label_masks_and_params_local = gen_synthetic_overseg(sc, num_samples=num_samples, peak_distance=10, num_gauss_areas=num_gauss_area, edt_gauss_center_val=10, gauss_std=16, marker_method="local",  gauss_center_val=150)
        sc.uns[overseg_uns_key].extend(label_masks_and_params_hmax)
        sc.uns[overseg_uns_key].extend(label_masks_and_params_local)
    return sc

from multiprocessing import Pool
pool = Pool()
res_single_cells = []
for res_sc in tqdm(pool.imap_unordered(process_sc_synthetic_overseg_crops, single_cells, chunksize=1), total=len(single_cells)):
    res_single_cells.append(res_sc)
# pool.map(process_sc_synthetic_overseg_crops, single_cells)
pool.close()
pool.join()

In [ ]:
single_cells = res_single_cells

In [ ]:
for i in range(5):
    sc = res_single_cells[i]

    # somehow failed to generate synthetic data meeting our criterion
    if len(sc.uns[overseg_uns_key]) == 0:
        continue
    for j in range(len(sc.uns[overseg_uns_key])):
        label_mask, eroded_label_mask, params = sc.uns[overseg_uns_key][j]
        print(params)
        fig, axes = plt.subplots(1, 2, figsize=(5, 2))
        ax = axes[0]
        ax.imshow(label_mask)
        ax.set_title("label_mask")
        ax = axes[1]
        ax.imshow(eroded_label_mask)
        ax.set_title("eroded_label_mask")
        plt.show()

Remove overseg out directory contents if necessary
```# !rm -r $overseg_out_dir```

In [ ]:
len(single_cells)
np.sum([len(sc.uns[overseg_uns_key]) for sc in single_cells])

In [ ]:
from livecellx.segment.ou_utils import csn_augment_helper
subdir = Path("synthetic_overseg")
overseg_out_dir = out_dir / subdir
raw_out_dir = overseg_out_dir / "raw"

# seg_out_dir is the directory containing all raw segmentation masks for training
# e.g. the eroded raw segmentation masks
seg_out_dir = overseg_out_dir / "seg"

# raw_seg_dir is the directory containing all raw segmentation masks for recording purposes
raw_seg_dir = overseg_out_dir / "raw_seg_crop"
gt_out_dir = overseg_out_dir / "gt"
gt_label_out_dir = overseg_out_dir / "gt_label_mask"
augmented_seg_dir = overseg_out_dir / "augmented_seg"
raw_transformed_img_dir = overseg_out_dir / "raw_transformed_img"
augmented_diff_seg_dir = overseg_out_dir / "augmented_diff_seg"
meta_path = overseg_out_dir / "metadata.csv"
syn_overseg_df_save_path = overseg_out_dir / "data.csv"

os.makedirs(raw_out_dir, exist_ok=True)
os.makedirs(seg_out_dir, exist_ok=True)
os.makedirs(raw_seg_dir, exist_ok=True)
os.makedirs(gt_out_dir, exist_ok=True)
os.makedirs(augmented_seg_dir, exist_ok=True)
os.makedirs(gt_label_out_dir, exist_ok=True)
os.makedirs(raw_transformed_img_dir, exist_ok=True)
os.makedirs(augmented_diff_seg_dir, exist_ok=True)


overseg_train_path_tuples = []
augmented_overseg_data = []
filename_pattern = "img-%d_syn-%d.tif"
overseg_metadata = []
overseg_erosion_scale_factors = np.linspace(-0.1, 0, 10)
all_df = None
for sc in tqdm(single_cells):
    img_id = sc.timeframe
    for syn_id, overseg_datarow in enumerate(sc.uns[overseg_uns_key]):
        params = overseg_datarow[1]
        img_crop = sc.get_contour_img()
        raw_seg_crop = overseg_datarow[0]
        eroded_seg_crop = overseg_datarow[1]

        combined_gt_label_mask = sc.get_contour_mask()
        assert img_crop.shape == raw_seg_crop.shape == combined_gt_label_mask.shape
        raw_img_path = raw_out_dir / (filename_pattern % (img_id, syn_id))
        seg_img_path = seg_out_dir / (filename_pattern % (img_id, syn_id))
        raw_seg_img_path = raw_seg_dir / (filename_pattern % (img_id, syn_id))
        gt_img_path = gt_out_dir / (filename_pattern % (img_id, syn_id))
        gt_label_img_path = gt_label_out_dir / (filename_pattern % (img_id, syn_id))

        # metadata is a dict, containing params used to genereate our synthetic overseg data
        meta_info = overseg_datarow[2]
        meta_info["raw_img_path"] = raw_img_path
        meta_info["seg_img_path"] = seg_img_path
        meta_info["gt_img_path"] = gt_img_path
        
        overseg_metadata.append(meta_info)

        # call csn augment helper
        res_dict = csn_augment_helper(img_crop=img_crop, 
            seg_label_crop=eroded_seg_crop, 
            combined_gt_label_mask=combined_gt_label_mask,
            overseg_raw_seg_crop=raw_seg_crop,
            overseg_raw_seg_img_path=raw_seg_img_path,
            scale_factors=overseg_erosion_scale_factors,
            train_path_tuples=overseg_train_path_tuples,
            augmented_data=augmented_overseg_data,
            img_id=img_id,
            seg_label=syn_id,
            gt_label=sc.timeframe,
            raw_img_path=raw_img_path,
            seg_img_path=seg_img_path,
            gt_img_path=gt_img_path,
            gt_label_img_path=gt_label_img_path,
            augmented_seg_dir=augmented_seg_dir,
            augmented_diff_seg_dir=augmented_diff_seg_dir,
            filename_pattern=filename_pattern,
            raw_transformed_img_dir=raw_transformed_img_dir,
            # df_save_path=syn_overseg_df_save_path,
        )
        all_df = res_dict["df"]

all_df.to_csv(syn_overseg_df_save_path, index=False)


In [ ]:
pd.DataFrame(
    overseg_train_path_tuples,
    columns=["raw", "seg", "gt", "raw_seg", "scale", "aug_diff_mask", "gt_label_mask", "raw_transformed_img"],
).to_csv(overseg_out_dir / "data.csv", index=False)

pd.DataFrame(
    overseg_metadata,
).to_csv(meta_path, index=False)


## Generate Synthetic Undersegmentation data

In [ ]:
def combine_sc(sc1, sc2):
    pass
sc1 = single_cells[0]
sc2 = single_cells[1]
sc2.show(padding=10)
plt.show()
sc2.show_contour_img()

In [ ]:
def generate_underseg_cases(scs):
    pass



## Combine data.csv files generated in each subfolder

In [ ]:
dataframes = []
for subdir in out_dir.iterdir():
    if subdir.is_dir():
        data_path = subdir / "data.csv"
        dataframe = pd.read_csv(data_path)
        dataframe["subdir"] = subdir.name
        dataframes.append(dataframe)
combined_dataframe = pd.concat(dataframes)
combined_dataframe.to_csv(out_dir / "train_data.csv", index=False)

In [ ]:
len(combined_dataframe)

In [ ]:
combined_dataframe[:2]

In [ ]:
def match_overseg_mask_labels(mask: np.array, sc: SingleCellStatic, io_seg_threshold=0.6):
    """match a single cell's label in a mask. sc must contain its contour information.

    Parameters
    ----------
    mask : np.array
        _description_
    sc : 
        _description_

    Returns
    -------
    a dict with keys "seg_label" and "iou"
    """    
    cropped_seg_mask = SingleCellStatic.gen_skimage_bbox_img_crop(sc.bbox, seg_mask)

    # Note the order of the arguments are reversed compared with underseg case
    # because match_mask_labels_by_iou is designed to match one seg mask to several gt masks (select gt sc's best matching seg label)
    # and in the oversegmentation case, we need to match one gt mask to several seg masks
    _, seg2gt_info = match_mask_labels_by_iou(seg_mask=sc.get_contour_mask(), gt_mask=cropped_seg_mask, return_all=True)
    matched_seg_labels = []
    for info in seg2gt_info:
        seg_label = info["seg_label"]
        iou = info["iou"]
        # Not the code below is not a bug: remember we reverse the order of the arguments
        io_gt = info["io_seg"]
        io_seg = info["io_gt"]
        if io_seg > io_seg_threshold:
            matched_seg_labels.append(seg_label)
    return matched_seg_labels

for sc in tqdm(single_cells):
    sc_img_path_substr = os.path.basename(sc.meta["path"]).split(".")[0]
    seg_mask = seg_data.get_img_by_url(sc_img_path_substr, substr=True)
    sc.meta["overseg_labels"] = match_seg_mask_label(seg_mask, sc)["seg_label"]
    sc.mask_dataset = seg_data

## TODO Below: adapt to new code (for real data's oversegmentation case); Skip for now because there is no oversegmentation case in our dataset

In [ ]:
(img_crop, seg_crop, combined_mask, aug_mask, aug_diff_mask, combined_gt_label_mask, raw_transformed_img_crop) = augmented_data[(1, 14)][0]

In [ ]:
np.unique(combined_mask), np.unique(aug_mask)

In [ ]:
import matplotlib.patches as mpatches

mask_tuples = list(augmented_overseg_data.values())[0]
nsamples = 10
# nsamples = np.sum([len(v) for v in augmented_data.values()])
ncol = 7
fig, axes = plt.subplots(nsamples, ncol, figsize=(25, 5.5 * nsamples))
fig.tight_layout()

sample_counter = 0
for key, mask_tuples in augmented_overseg_data.items():
    for j in range(len(mask_tuples)):
        i = sample_counter
        axes[i, 0].imshow(mask_tuples[j][0])
        axes[i, 0].set_title("raw mask")
        axes[i, 1].imshow(mask_tuples[j][1])
        axes[i, 1].set_title("seg img")
        axes[i, 2].imshow(mask_tuples[j][2])
        axes[i, 2].set_title("combined mask")
        axes[i, 3].imshow(mask_tuples[j][3])
        axes[i, 3].set_title("augmented mask")
        im = axes[i, 4].imshow(mask_tuples[j][4])
        axes[i, 4].set_title("augmented diff mask")

        values = np.unique(mask_tuples[j][4])
        colors = [ im.cmap(im.norm(value)) for value in values]
        patches = [ mpatches.Patch(color=colors[i], label="Level {l}".format(l=values[i]) ) for i in range(len(values)) ]
        axes[i, 4].legend(handles=patches, loc=2, borderaxespad=0. )

        axes[i, 5].imshow(mask_tuples[j][5])
        axes[i, 5].set_title("combined gt label mask")

        axes[i, 6].imshow(mask_tuples[j][6])
        axes[i, 6].set_title("raw transformed img")

        sample_counter += 1
        if sample_counter >= nsamples:
            break
    if sample_counter >= nsamples:
        break

Deprecated code for augmenting a single cell for CSN
```
from pathlib import Path
from copy import deepcopy
from PIL import Image, ImageSequence

img_paths = []
augment_dir = Path("../datasets/a549_ccnn/augmented_cells")
augmented_cells = []
scale_factor = 0.01
padding = 100

def augment_single_cell_csn(sc: SingleCellStatic, augment_dir: Path, scale_factor=0.01, padding=40, show=False, bg_value=0):
    mask = sc.get_mask_crop(padding=padding).astype(np.uint8)
    mask_label = sc.meta["seg_label"]
    mask[mask!=mask_label] = bg_value

    augmented_mask = dilate_or_erode_mask(mask, scale_factor=scale_factor)
    augmentation_save_path = augment_dir / f"{i}_mask.tif"
    save_png(augmented_mask, str(augmentation_save_path))

    meta = {}
    meta["original_path"] = sc.meta["path"]
    meta["original_seg_label"] = sc.meta["seg_label"]
    meta["path"] = augmentation_save_path.as_posix()
    meta["padding"] = padding
    meta["scale_factor"] = scale_factor
    meta["method"] = "dilate/erode"

    new_sc = SingleCellStatic(
        img_dataset=sc.img_dataset,
        mask_dataset=sc.mask_dataset,
        contour = np.array(sc.contour),
        meta=meta,
    )

    if show:
        fig, axes = plt.subplots(1, 4, figsize=(10, 10))
        # sc.show_mask(ax=axes[0])
        sc.show(ax=axes[0])
        axes[2].imshow(mask)
        sc.show_contour_img(ax=axes[1])
        axes[3].imshow(augmented_mask)

        axes[0].set_title("original image crop")
        axes[2].set_title("seg mask")
        axes[1].set_title("gt contour mask")
        axes[3].set_title("augmented mask")
    return new_sc


augmented_single_cells = []
for i in tqdm(range(len(single_cells))):
    sc = single_cells[i]
    augmented_sc = augment_single_cell_csn(sc, augment_dir, scale_factor=scale_factor, padding=padding, show=False)
    augmented_single_cells.append(augmented_sc)
```


Undersegmentation case:
    overlay all the ground truth masks on the original image, and then use the result as the ground truth mask for the undersegmentation case.
    Segmentation mask 
    gt mask

OVERSEGMENTATION CASE:
    # future work
    Segmentation mask
    gt mask


1. Overlay segmentation mask and gt mask
2. Shift the cells in the segmentation mask to the center of the cell
3. Synthetic dataset

thin_plate_spline?